**--- Part One ---**

In [12]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
# case = "jfm"
# case = "gatton"

# verbose output
verbose = False

import numpy as np
from itertools import cycle

class OverlapError(Exception):
    pass

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

gusts = cycle(input_lines[0])


v_offset_arr = np.zeros((3,7),int)

class Rock():
    def __init__(self,shape):
        self.shape = shape#[::-1]
        self.x = 2
        self.y = -1
        self.width = len(shape[0])
        self.height = len(shape)

class Cave():
    def __init__(self):
        self.width = 7
        self.cur_height = 0
        # initialize empty cave
        self.pile_array = np.empty((0,7),int)
        self.pile_list = self.pile_array.tolist()
        self.cur_rock = None
        self.verbose=None
        
    def add_rock(self, rock_shape: Rock, initial=True, verbose=None):
        if verbose:
            self.verbose = verbose
        if self.verbose: print('adding rock')
        self.cur_rock = rock_shape
        self.cur_rock.x = 2
        # Add rows of zeros for vertical offset plus room for the shape
        #rock = np.zeros((3+rock.height,7))
        # Add placeholders to top of pile
        self.rock = np.zeros((self.cur_rock.height+3,7),int)
        self.pile = np.zeros(self.rock.shape)
        self.place_rock(self.cur_rock,self.rock)
        ## self.pile = np.vstack((self.pile, np.zeros((rock_shape.height+3,7),int)))
        ## self.rock = np.zeros(self.pile.shape, int)
        #self.pile = self.pile + self.rock
        # place rock in temp pile
        if self.verbose: print(f'rock.x: {self.cur_rock.x}')
        self.place_rock(self.cur_rock,self.rock)
        if self.verbose: self.print_pile(self.rock)
        #if self.verbose: print(np.flipud(self.pile))
        #print(self.rock)
        #self.data = np.append(self.data,v_offset_arr)
        #self.final = np.append(self.final,self.temp)
        #self.place_rock(rock, rock.x, rock.y)
        #print(self.pile)
        self.jet_gust(next(gusts))
        while True:
            if self.check_clearance():
                self.move_down()
                self.jet_gust(next(gusts))
                #if self.verbose: print(np.flipud(self.rock))
            else:
                break
            # self.jet_gust(next(gusts))
            # if self.check_clearance():
            #     self.move_down()
        tmp = self.pad_array(self.rock, self.pile)
        self.pile = self.pile + tmp
        self.clean_zeros()
        #print('')
        #self.print_pile(self.pile)
 
    def place_rock(self, rock, pile):
        src_y = 1
        for row in rock.shape:
            src_x = 0
            dest_y = -src_y
            for val in row:
                dest_x = src_x + rock.x
                print('')
                print(pile)
                pile[dest_y,dest_x] += val
                src_x += 1
            src_y += 1

    def check_clearance(self):
        if self.verbose: 
            print('checking clearance')  
            print(self.pile.shape)
        if self.rock.shape[0] ==1:            
            if self.verbose: print('returning False - initialization')
            return False

        # if (self.pile[0,:].any()>0) and (self.rock[0,:].any()>0):
        #     if self.verbose: print('returning False - initialization')
        #     return False
        try:
            if self.verbose: print('in try')
            if self.verbose: print('temp pile')
            self.move_down(apply=False)
            return True
            # new_pile = self.pile[:-1,:] + self.rock[1:,:]
            # if self.verbose: print('future pile')
            # if self.verbose: self.print_pile(new_pile)
            # if new_pile.max()>1:
            #     if self.verbose: 
            #         print('returning False - overlap')
            #         print('temp_pile')
            #         print(self.rock)
            #         print('pile')
            #         print(self.pile)
            #         print('new pile')
            #         print(new_pile)
            #     return False
            # else:
                # if self.verbose: print('returning True')
                # return True
        except IndexError:
            if self.verbose: print('returning False due to index error')
            return False
        except OverlapError:
            if self.verbose: print('returning False due to overlap')
            return False

    def move_down(self,apply=True):
        if self.verbose: print('moving down')
        tmp = self.rock[1:,:]
        tmp = self.pad_array(tmp, self.pile)     
        # if (delta := self.pile.shape[0]-tmp.shape[0]):
        #     print(f'{delta=}')
        #     pad_temp_rows = np.zeros((delta,7),int)
        #     tmp = np.vstack((tmp,pad_temp_rows))
        #     print(tmp)
        new_pile = self.pile + tmp
        if not new_pile.max()>1:
            if apply:
                self.rock = self.rock[1:,:]
                #self.clean_zeros()
                # if self.pile[-1,:].all() == 0:
                #     self.pile = self.pile[:-1,:]
        else:
            raise OverlapError


    def print_pile(self,pile):
        def conv(x):
            if x==0:
                return '.'
            else:
                return '#'
        for row in np.flipud(pile):
            print(f'|{"".join([conv(x) for x in row])}|')
        print(f'+{"-"*7}+')
        #print(np.flipud(pile))
    
    def pad_array(self, small_arr, large_arr):
        if (delta := large_arr.shape[0]-small_arr.shape[0]):
            pad_temp_rows = np.zeros((delta,7),int)
            small_arr = np.vstack((small_arr,pad_temp_rows))
        return small_arr
        
    def jet_gust(self,gust):
        new_pile=None
        rock = self.cur_rock
        if gust == '>': # move right
            if self.verbose: print('moving right')
            if rock.x+rock.width<7:
                new_pile = np.zeros(self.rock.shape,int)
                self.cur_rock.x += 1
                self.place_rock(self.cur_rock, new_pile)
                tmp = self.pad_array(new_pile, self.pile)
                if not ((tmp+self.pile).max()>1):
                    self.rock=new_pile
                else: 
                    self.cur_rock.x -= 1
        elif gust == '<': # move left
            if self.verbose: print('moving left')
            if rock.x>0:
                new_pile = np.zeros(self.rock.shape,int)
                self.cur_rock.x -= 1
                self.place_rock(self.cur_rock, new_pile)
                tmp = self.pad_array(new_pile, self.pile)
                if not ((tmp+self.pile).max()>1):
                    self.rock=new_pile
                else:
                    self.cur_rock.x += 1

        if self.verbose: print('after gust')
        if self.verbose: self.print_pile(self.rock)
       
    
    def clean_zeros(self):
        self.pile = self.pile[~np.all(self.pile==0,axis=1)]
        self.pile = np.where(self.pile>0,1,0)
        

rock_shapes = [
    [[1,1,1,1]],
    [[0,1,0],[1,1,1],[0,1,0]],
    [[0,0,1],[0,0,1],[1,1,1]],
    [[1],[1],[1],[1]],
    [[1,1],[1,1]]
]

rocks = cycle([Rock(shape) for shape in rock_shapes])

N_rocks = 2022

N_vert_offset = 3

stack = np.zeros(7)+1

# First Rock
#
# init stack

stack

array([1., 1., 1., 1., 1., 1., 1.])

In [11]:
cave = Cave()
gusts = cycle(input_lines[0])
rocks = cycle([Rock(shape) for shape in rock_shapes])
print(cave.pile_list) 
n = 2022
for i in range(n):
    #print(f'rock {i}')
    if i==n:
        cave.add_rock(next(rocks),verbose=True)
    else:
        cave.add_rock(next(rocks),verbose=False)
    #cave.print_pile(cave.pile)
#cave.print_pile(cave.pile)
#for i in range(2022):
#    cave.add_rock(next(rocks))
#cave.print_pile(cave.pile)
print(cave.pile.shape)
# cave.add_rock(next(rocks))
# cave.add_rock(next(rocks))
# cave.add_rock(next(rocks))
# cave.add_rock(next(rocks))
# #print(cave.temp_pile)
# cave.jet_gust('>')
# print(cave.temp_pile)
# cave.jet_gust('<')
# cave.jet_gust('<')
# print(cave.temp_pile)
# print(cave.check_clearance())
# cave.move_down()
# print(cave.pile)
# cave.move_down()
# print(cave.pile)
# cave.check_clearance()
# cave.move_down()
# print(cave.pile)

[]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 2 1 1 1 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 2 2 1 1 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 2 2 2 1 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0]]

[[0 0 0 0 0 0 0]]

[[0 0 1 0 0 0 0]]

[[0 0 1 1 0 0 0]]

[[0 0 1 1 1 0 0]]

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 

IndexError: index -3 is out of bounds for axis 0 with size 2

In [383]:
cave.print_pile(cave.pile)

|.....#.|
|....###|
|.#####.|
|...##..|
|...##..|
|.##.###|
|.##.###|
|..#####|
|.###..#|
|..#####|
|..#.##.|
|..#.##.|
|..#..#.|
|..#..#.|
|..####.|
|.###...|
|..#....|
|..####.|
|..##...|
|..##...|
|..###.#|
|..#...#|
|.###..#|
|..#...#|
|#####.#|
|...####|
|....#.#|
|..#####|
|..###.#|
|.####.#|
|###.###|
|.#####.|
|.##....|
|.##....|
|..#....|
|..#...#|
|..#...#|
|..#####|
|..###..|
|...#...|
|...####|
|...##..|
|...##..|
|..####.|
|..#.#..|
|..####.|
|..#.#..|
|..####.|
|...##..|
|...##..|
|...#...|
|...#...|
|...#.#.|
|...#.#.|
|...###.|
|...#...|
|..###..|
|...#...|
|..####.|
|...##..|
|...##..|
|.###.#.|
|...#.#.|
|..####.|
|...#.#.|
|####.##|
|..#..#.|
|####.#.|
|..####.|
|...#.#.|
|..####.|
|....##.|
|....##.|
|..#.###|
|..#####|
|..##..#|
|..##..#|
|..#...#|
|###...#|
|..#...#|
|.###..#|
|..#####|
|..##...|
|..###.#|
|...####|
|#####.#|
|###..##|
|###.###|
|.#####.|
|.##....|
|####...|
|#.###..|
|#..#...|
|#####..|
|##.....|
|##.....|
|.##...#|
|.##.###|
|.##..#.|
|.##.###|


In [6]:
from rich import inspect
inspect(r1)
r1.place_rock(0,0,r1.cave)
print(np.flipud(r1.cave))

╭──────────── <class '__main__.Rock'> ─────────────╮
│ ╭──────────────────────────────────────────────╮ │
│ │ <__main__.Rock object at 0x00000252D0D7A100> │ │
│ ╰──────────────────────────────────────────────╯ │
│                                                  │
│ height = 1                                       │
│    loc = [3, 2]                                  │
│  shape = [[1, 1, 1, 1]]                          │
│  width = 4                                       │
│      x = 2                                       │
│      y = 3                                       │
╰──────────────────────────────────────────────────╯

AttributeError: 'Rock' object has no attribute 'place_rock'

In [78]:
r1.move('>')
print(np.flipud(r1.cave))


[[0. 0. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [141]:
cave = 
blank_lines = [[0 for i in range(7)] for j in range(3)]
v_offset_arr = np.zeros((3,7),int)
print(v_offset_arr)
ra_shape = np.array([[0,0,1,1,1,1,0]])
ra_shape.shape
ra = np.append(v_offset_arr, ra_shape,axis=0)
print(np.flipud(ra))


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 1 1 1 1 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [144]:
cave = np.empty((0,7),int)
cave = np.vstack()
cave = np.append(cave, np.array())

TypeError: _vhstack_dispatcher() missing 1 required positional argument: 'tup'

[[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]]